# Quant Analyzer

This notebook showcases a working code example of how to use AIMET to apply Quant Analyzer.
Quant Analyzer is a feature which performs various analyses on a model to understand how each layer in the model responds to quantization.

#### Overall flow
This notebook covers the following
1. Instantiate the example evaluation pipeline
2. Load the FP32 model
3. Apply QuantAnalyzer to the model


#### What this notebook is not
* This notebook is not designed to show state-of-the-art results.
* For example, it uses a relatively quantization-friendly model like Resnet18.
* Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly.

---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification. If you already have a version of the dataset readily available, please use that. Else, please download the dataset from appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The ImageNet dataset typically has the following characteristics and the dataloader provided in this example notebook rely on these
- Subfolders 'train' for the training samples and 'val' for the validation samples. Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset. E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. But for the purpose of running this notebook, you could perhaps reduce the dataset to say 2 samples per class. This exercise is left upto the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [1]:
DATASET_DIR = '/path/to/dataset/'         # Please replace this with a real directory

---

## 1. Example evaluation and training pipeline

The following is an example training and validation loop for this image classification task.

- **Does AIMET have any limitations on how the training, validation pipeline is written?**

    Not really. We will see later that AIMET will modify the user's model to create a QuantizationSim model which is still a TensorFlow model.
    This QuantizationSim model can be used in place of the original model when doing inference or training.

- **Does AIMET put any limitation on the interface of the evaluate() or train() methods?**

    Not really. You should be able to use your existing evaluate and train routines as-is.

In [2]:
import cv2
import os
import torch
import json
import numpy as np
from tqdm.notebook import tqdm
from copy import deepcopy

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from mmcv.transforms import Compose
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def read_txt(txt_path):
    with open(txt_path) as f:
        data = f.readlines()
    data = [x.strip() for x in data]
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

In [3]:
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 1000
BATCH_SIZE = 64

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)
eval_data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)
calibration_images = read_txt('/teamspace/studios/this_studio/aimet/Examples/torch/quantization/calibration_image_ids.txt')
calibration_data_loader = DataLoader(calibration_images, batch_size=BATCH_SIZE)

DEVICE

[2024-09-06 20:38:23,618] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

09/06 20:38:27 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


device(type='cuda', index=0)

In [4]:
from collections import OrderedDict
from copy import deepcopy

m = deepcopy(model.model)

def is_leaf(module): 
    return len(module._modules) == 0

def replace_bn(m):

    if is_leaf(m):
        return 

    for _, child in m.named_children(): 
        
        if "bn" in child._modules.keys():
            bn = child._modules.get("bn")
            bn_params = deepcopy(bn._parameters)
            bn_buffers = deepcopy(bn._buffers)
            new_bn = torch.nn.BatchNorm2d(bn.num_features, eps=bn.eps, momentum=bn.momentum, affine=bn.affine, track_running_stats=bn.track_running_stats)
            new_bn._parameters["weight"].data = bn_params["weight"].data
            new_bn._parameters["bias"].data = bn_params["bias"].data
            new_bn._buffers["running_mean"].data = bn_buffers["running_mean"].data
            new_bn._buffers["running_var"].data = bn_buffers["running_var"].data
            new_bn._buffers["num_batches_tracked"].data = bn_buffers["num_batches_tracked"].data
            child._modules["bn"] = new_bn
            
        replace_bn(child)


In [5]:
from tqdm.notebook import tqdm
import torch

from mmdet.models.utils import samplelist_boxtype2tensor
from mmengine.registry import MODELS
from mmcv.transforms import Compose

test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

collate_preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True

preprocessor = MODELS.build(model.cfg.model.data_preprocessor)
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

loading annotations into memory...
Done (t=1.00s)
creating index...
index created!


In [6]:
def pass_calibration_data(model: torch.nn.Module, samples: int):
    data_loader = eval_data_loader
    batch_size = data_loader.batch_size
    model.eval()
    batch_ctr = 0
    with torch.no_grad():
        for image_path in tqdm(calibration_data_loader):
            image_path = [os.path.join(IMAGES_DIR, x) for x in image_path]
            pre_processed = collate_preprocessor(inputs=image_path, batch_size=batch_size)
            _, data = list(pre_processed)[0]
            data = preprocessor(data, False)
            
            preds = model(data['inputs'].to(DEVICE))  

            # batch_ctr += 1
            # if (batch_ctr * batch_size) > samples:
            #     break

In [7]:
from glob import glob
from mmengine.structures import InstanceData

def eval_callback(model, save_dir="./temp"):
    data_loader = eval_data_loader
    new_preds = []
    model.eval()
    with torch.no_grad():
        for image_id, image_path, _ in tqdm(data_loader):
            pre_processed = collate_preprocessor(inputs=image_path, batch_size=BATCH_SIZE)
            _, data = list(pre_processed)[0]
            data = preprocessor(data, False)
            preds = model(data['inputs'].to(DEVICE))
            batch_img_metas = [
            data_samples.metainfo for data_samples in data['data_samples']
            ]
            preds = predict_by_feat(*preds, batch_img_metas=batch_img_metas, rescale=True)
            preds = add_pred_to_datasample(data['data_samples'], preds)
            
            for img_id, pred in zip(image_id, preds):
                pred = pred.pred_instances
                new_pred = InstanceData(metainfo={"img_id": int(img_id)})
                new_pred.bboxes = [np.array(p) for p in pred['bboxes'].cpu()]
                new_pred.labels = pred['labels'].cpu()
                new_pred.scores = pred['scores'].cpu()
                new_preds.append(new_pred)

    eval_results = test_evaluator(new_preds)
    
    os.makedirs(save_dir, exist_ok=True)
    num_file = len(glob(f"{save_dir}/eval_acc_*"))
    with open(f"{save_dir}/eval_acc_{num_file}.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    bbox_map = eval_results['bbox_mAP']
    return bbox_map

In [8]:
model = deepcopy(model.model)
replace_bn(model)

In [9]:
dict(model.named_modules())['backbone.stem.0']

ConvModule(
  (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activate): SiLU(inplace=True)
)

AIMET quantization simulation requires the user's model definition to follow certain guidelines.
For example, functionals defined in forward pass should be changed to equivalent torch.nn.Module.
AIMET user guide lists all these guidelines.

The following **ModelPreparer** API uses new graph transformation feature available in PyTorch 1.9+ version and automates model definition changes required to comply with the above guidelines.

In [10]:
from aimet_torch.model_preparer import prepare_model

model = prepare_model(model)

2024-09-06 20:38:33,501 - root - INFO - AIMET
2024-09-06 20:38:33,670 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.module_add} 
2024-09-06 20:38:33,671 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.module_cat} 
2024-09-06 20:38:33,673 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.attention.module_mul} 
2024-09-06 20:38:33,674 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.blocks.0.module_add_1} 
2024-09-06 20:38:33,677 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.module_cat_1} 
2024-09-06 20:38:33,682 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage2.1.attention.module_mul_1} 
2024-09-06 20:38:33,684 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage3.1.blocks.0.modu

---
We should decide whether to place the model on a CPU or CUDA device.
This example code will use CUDA if available in your current execution environment.
You can change this logic and force a device placement if needed.

In [11]:
model.to(DEVICE)

GraphModule(
  (backbone): Module(
    (stem): Module(
      (0): Module(
        (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
      (1): Module(
        (conv): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      )
      (2): Module(
        (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): CustomSiLU(
          (sigmoid): Sigmoid()
          (mul): Multiply()
        )
      

---

## 3. Apply QuantAnalyzer to the model

QuantAnalyzer requires two functions to be defined by the user for passing data through the model:

**Forward pass callback**

One function will be used to pass representative data through a quantized version of the model to calibrate quantization parameters.
This function should be fairly simple - use the existing train or validation data loader to extract some samples and pass them to the model.
We don't need to compute any loss metrics, so we can just ignore the model output.

The function **must** take two arguments, the first of which will be the model to run the forward pass on.
The second argument can be anything additional which the function requires to run, and can be in the form of a single item or a tuple of items.

If no additional argument is needed, the user can specify a dummy "_" parameter for the function.

A few pointers regarding the forward pass data samples:

- In practice, we need a very small percentage of the overall data samples for computing encodings.
  For example, the training dataset for ImageNet has 1M samples. For computing encodings we only need 500 to 1000 samples.
- It may be beneficial if the samples used for computing encoding are well distributed.
  It's not necessary that all classes need to be covered since we are only looking at the range of values at every layer activation.
  However, we definitely want to avoid an extreme scenario like all 'dark' or 'light' samples are used - e.g. only using pictures captured at night might not give ideal results.

The following shows an example of a routine that passes unlabeled samples through the model for computing encodings.
This routine can be written in many ways; this is just an example.
This function only requires unlabeled data as no loss or other evaluation metric is needed.

In order to pass this function to QuantAnalyzer, we need to wrap it in a CallbackFunc object, as shown below.
The CallbackFunc takes two arguments: the callback function itself, and the inputs to pass into the callback function.

In [12]:
from aimet_torch.quant_analyzer import CallbackFunc

forward_pass_callback = CallbackFunc(pass_calibration_data, CALIBRATION_DATASET_SIZE)

---

**Evaluation callback**

The second function will be used to evaluate the model, and needs to return an accuracy metric.
In here, the user should pass any amount of data through the model which they would like when evaluating their model for accuracy.

Like the forward pass callback, this function also must take exactly two arguments: the model to evaluate, and any additional argument needed for the function to work.
The second argument can be a tuple of items in case multiple items are needed.

We will be using the ImageNetDataPipeline's evaluate defined above for this purpose.
Like the forward pass callback, we need to wrap the evaluation callback in a CallbackFunc object as well.

In [13]:
eval_callback = CallbackFunc(eval_callback, "./folded_bn_quant_analyzer_stats")

---

**Enabling MSE loss per layer analysis**

An optional analysis step in QuantAnalyzer calculates the MSE loss per layer in the model, comparing the layer outputs from the original FP32 model vs. a quantized model.
To perform this step, the user needs to also provide an unlabeled DataLoader to QuantAnalyzer.

We will demonstrate this step by using the ImageNetDataLoader imported above.

In [14]:
# data_loader = ImageNetDataPipeline.get_val_dataloader()

---

QuantAnalyzer also requires a dummy input to the model.
This dummy input does not need to be representative of the dataset.
All that matters is that the input shape is correct for the model to run on.

In [15]:
dummy_input = torch.rand(1, 3, 640, 640).to(DEVICE)   # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)

---
We are now ready to apply QuantAnalyzer.

In [16]:
from aimet_torch.quant_analyzer import QuantAnalyzer

quant_analyzer = QuantAnalyzer(model, dummy_input, forward_pass_callback, eval_callback, results_dir="./quant_analyzer_int32")

To enable the MSE loss analysis, we set the following:

In [17]:
# quant_analyzer.enable_per_layer_mse_loss(data_loader, num_batches=4)

Finally, to start the analyzer, we call .analyze().

A few of the parameters are explained here:
- **quant_scheme**:
    - We set this to "post_training_tf_enhanced"
      With this choice of quant scheme, AIMET will use the TF Enhanced quant scheme to initialize the quantization parameters like scale/offset.
- **default_output_bw**: Setting this to 8 means that we are asking AIMET to perform all activation quantizations in the model using integer 8-bit precision.
- **default_param_bw**: Setting this to 8 means that we are asking AIMET to perform all parameter quantizations in the model using integer 8-bit precision.

There are other parameters that are set to default values in this example.
Please check the AIMET API documentation of QuantizationSimModel to see reference documentation for all the parameters.

When you call the analyze method, the following analyses are run:

- Compare fp32 accuracy, accuracy with only parameters quantized, and accuracy with only activations quantized
- For each layer, track the model accuracy when quantization for all other layers is disabled (enabling quantization for only one layer in the model at a time)
- For each layer, track the model accuracy when quantization for all other layers is enabled (disabling quantization for only one layer in the model at a time)
- Track the minimum and maximum encoding parameters calculated by each quantizer in the model as a result of forward passes through the model with representative data
- When the TF Enhanced quantization scheme is used, track the histogram of tensor ranges seen by each quantizer in the model as a result of forward passes through the model with representative data
- If enabled, track the MSE loss seen at each layer by comparing layer outputs of the original fp32 model vs. a quantized model

In [27]:
# MODULES TO IGNORE AFTER FOLDING
modules_to_change = ["backbone.stage2.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage1.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv", "backbone.stage3.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage4.2.blocks.0.conv2.depthwise_conv.conv", "neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.conv", "neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.conv", "neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.conv", "neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.conv"]

# # MODULES TO IGNORE BEFORE FOLDING
# modules_to_change = ['backbone.stage2.1.blocks.0.conv2.depthwise_conv.bn', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.bn', 'backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage3.1.blocks.0.conv2.depthwise_conv.bn', 'backbone.stage4.2.blocks.0.conv2.depthwise_conv.bn', 'neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.bn', 'neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.bn', 'neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.bn', 'neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.bn']

len(modules_to_change)

9

In [31]:
from aimet_torch.quantsim import QuantizationSimModel
from aimet_common.defs import QuantScheme, QuantizationDataType

modules = dict(model.named_modules())
modules_to_change = ["backbone.stage2.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage1.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv", "backbone.stage3.1.blocks.0.conv2.depthwise_conv.conv", "backbone.stage4.2.blocks.0.conv2.depthwise_conv.conv", "neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.conv", "neck.top_down_blocks.1.blocks.0.conv2.depthwise_conv.conv", "neck.bottom_up_blocks.0.blocks.0.conv2.depthwise_conv.conv", "neck.bottom_up_blocks.1.blocks.0.conv2.depthwise_conv.conv"]
modules_to_change = {modules[x]: {"output_bw": 32, "param_bw": 32, "data_type": QuantizationDataType.int, "module_name": x} for x in modules_to_change}

quant_sim = QuantizationSimModel(model, dummy_input, quant_scheme=QuantScheme.post_training_tf_enhanced, default_param_bw=8, default_output_bw=8, modules_to_change=modules_to_change, in_place=True)

quant_sim.compute_encodings(forward_pass_callback.func, forward_pass_callback.args)bb


2024-09-06 20:47:40,799 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-09-06 20:47:40,848 - Quant - INFO - Unsupported op type Squeeze
2024-09-06 20:47:40,849 - Quant - INFO - Unsupported op type Mean
2024-09-06 20:47:40,874 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


  0%|          | 0/16 [00:00<?, ?it/s]

In [33]:
print(str(quant_sim))

-------------------------
Quantized Model Report
-------------------------
----------------------------------------------------------
Layer: backbone.stem.0.conv
  Input[0]: Not quantized
  -------
  Param[weight]: bw=8, encoding-present=True
    StaticGrid TensorQuantizer:
    quant-scheme:QuantScheme.post_training_tf_enhanced, round_mode=RoundingMode.ROUND_NEAREST, bitwidth=8, enabled=True
    min:-1.6382535696029663, max=1.6254547834396362, delta=0.012798856012523174, offset=-128.0
  -------
  Param[bias]: Not quantized
  -------
  Output[0]: bw=8, encoding-present=True
    StaticGrid TensorQuantizer:
    quant-scheme:QuantScheme.post_training_tf_enhanced, round_mode=RoundingMode.ROUND_NEAREST, bitwidth=8, enabled=True
    min:-9.78048324584961, max=18.24224853515625, delta=0.1098930686712265, offset=-89.0
  -------
----------------------------------------------------------
Layer: backbone.stem.0.activate.sigmoid
  Input[0]: Not quantized
  -------
  Output[0]: bw=8, encoding-presen

In [19]:
from aimet_common.defs import QuantScheme, QuantizationDataType

modules = dict(model.named_modules())
modules_to_change = {modules[x]: {"output_bw": 32, "param_bw": 32, "data_type": QuantizationDataType.int, "module_name": x} for x in modules_to_change}

quant_analyzer.analyze(quant_scheme=QuantScheme.post_training_tf_enhanced,
                       default_param_bw=8,
                       default_output_bw=8,
                       config_file=None,
                       results_dir="./quant_analyzer_int32",
                       modules_to_change=modules_to_change,
                       model_in_place=True)

2024-09-06 20:38:54,561 - BatchNormFolding - INFO - 0 BatchNorms' weights got converted
2024-09-06 20:38:54,564 - QuantAnalyzer - INFO - Folded 76 batch norms
2024-09-06 20:38:59,097 - Quant - INFO - Changed quantization params for this module: backbone.stage1.1.blocks.0.conv2.depthwise_conv.conv
2024-09-06 20:38:59,107 - Quant - INFO - Changed quantization params for this module: backbone.stage2.1.blocks.0.conv2.depthwise_conv.conv
2024-09-06 20:38:59,113 - Quant - INFO - Changed quantization params for this module: backbone.stage2.1.blocks.0.conv2.pointwise_conv.conv
2024-09-06 20:38:59,119 - Quant - INFO - Changed quantization params for this module: backbone.stage3.1.blocks.0.conv2.depthwise_conv.conv
2024-09-06 20:38:59,127 - Quant - INFO - Changed quantization params for this module: backbone.stage4.2.blocks.0.conv2.depthwise_conv.conv
2024-09-06 20:38:59,135 - Quant - INFO - Changed quantization params for this module: neck.top_down_blocks.0.blocks.0.conv2.depthwise_conv.conv
20

  0%|          | 0/16 [00:00<?, ?it/s]

2024-09-06 20:41:01,046 - QuantAnalyzer - INFO - Exported per layer stats histogram plot(s).


AIMET will also output .html plots and json files where appropriate for each analysis to help visualize the data.

The following output files will be produced, in a folder specified by the user:
Output directory structure will be like below

```
results_dir
|-- per_layer_quant_enabled.html
|-- per_layer_quant_enabled.json
|-- per_layer_quant_disabled.html
|-- per_layer_quant_disabled.json
|-- min_max_ranges
|   |-- activations.html
|   |-- activations.json
|   |-- weights.html
|   +-- weights.json
|-- activations_pdf
|   |-- name_{input/output}_{index_0}.html
|   |-- name_{input/output}_{index_1}.html
|   |-- ...
|   +-- name_{input/output}_{index_N}.html
|-- weights_pdf
|   |-- layer1
|   |   |-- param_name_{channel_index_0}.html
|   |   |-- param_name_{channel_index_1}.html
|   |   |-- ...
|   |   +-- param_name_{channel_index_N}.html
|   |-- layer2
|   |   |-- param_name_{channel_index_0}.html
|   |   |-- param_name_{channel_index_1}.html
|   |   |-- ...
|   |   +-- param_name_{channel_index_N}.html
|   |-- ...
|   |-- layerN
|   |   |-- param_name_{channel_index_0}.html
|   |   |-- param_name_{channel_index_1}.html
|   |   |-- ...
|   +-- +-- param_name_{channel_index_N}.html
|-- per_layer_mse_loss.html
+-- per_layer_mse_loss.json
```

#### Per-layer analysis by enabling/disabling quantization wrappers

- per_layer_quant_enabled.html: A plot with layers on the x-axis and model accuracy on the y-axis, where each layer's accuracy represents the model accuracy when all quantizers in the model are disabled except for that layer's parameter and activation quantizers.
- per_layer_quant_enabled.json: A json file containing the data shown in per_layer_quant_enabled.html, associating layer names with model accuracy.
- per_layer_quant_disabled.html: A plot with layers on the x-axis and model accuracy on the y-axis, where each layer's accuracy represents the model accuracy when all quantizers in the model are enabled except for that layer's parameter and activation quantizers.
- per_layer_quant_disabled.json: A json file containing the data shown in per_layer_quant_disabled.html, associating layer names with model accuracy.

![per_layer_quant_enabled.html](./images/quant_analyzer_per_layer_quant_enabled.PNG)

#### Encoding min/max ranges

- min_max_ranges: A folder containing the following sets of files:
    - activations.html: A plot with output activations on the x-axis and min-max values on the y-axis, where each output activation's range represents the encoding min and max parameters computed during forward pass calibration (explained below).
    - activations.json: A json file containing the data shown in activations.html, associating layer names with min and max encoding values.
    - weights.html: A plot with parameter names on the x-axis and min-max values on the y-axis, where each parameter's range represents the encoding min and max parameters computed during forward pass calibration.
    - weights.json: A json file containing the data shown in weights.html, associating parameter names with min and max encoding values.

![min_max_ranges.html](./images/quant_analyzer_min_max_ranges.PNG)

#### PDF of statistics

- (If TF Enhanced quant scheme is used) activations_pdf: A folder containing html files for each layer, plotting the histogram of tensor values seen for that layer's output activation seen during forward pass calibration.
- (If TF Enhanced quant scheme is used) weights_pdf: A folder containing sub folders for each layer with weights.
  Each layer's folder contains html files for each parameter of that layer, with a histogram plot of tensor values seen for that parameter seen during forward pass calibration.

![weights_pdf.html](./images/quant_analyzer_weights_pdf.PNG)

#### Per-layer MSE loss
- (Optional, if per layer MSE loss is enabled) per_layer_mse_loss.html: A plot with layers on the x-axis and MSE loss on the y-axis, where each layer's MSE loss represents the MSE seen comparing that layer's outputs in the FP32 model vs. the quantized model.
- (Optional, if per layer MSE loss is enabled) per_layer_mse_loss.json: A json file containing the data shown in per_layer_mse_loss.html, associating layer names with MSE loss.

![per_layer_mse_loss.html](./images/quant_analyzer_per_layer_mse_loss.PNG)